In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wandb
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit

In [2]:
import sys
sys.path.append('../../../../code')

from metrics import get_hi_metrics

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/simon/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
train = pd.read_csv('../../../../data/hi/kdr/train_1.csv', index_col=0)
test = pd.read_csv('../../../../data/hi/kdr/test_1.csv', index_col=0)

train

,smiles,value
0,Brc1ccc(-c2nc3ccc(Nc4ccnc5ccccc45)cc3[nH]2)cc1,True
1064,CCc1ccc(-c2nc3ccc(Nc4ccnc5ccccc45)cc3[nH]2)cc1,False
1065,CCc1ccc(-c2nc3ccc(Nc4ncnc5ccccc45)cc3[nH]2)cc1,False
3722,COc1ccccc1-c1nc2ccc(Nc3ccnc4ccccc34)cc2[nH]1,False
4159,Cc1ccc(-c2nc3ccc(Nc4ccnc5ccccc45)cc3[nH]2)cc1,False
...,...,...
454,CC(C)S(=O)(=O)c1ccccc1Nc1nc(Nc2cccc(NC(=O)CN)c...,False
1850,COC(=O)c1cn2ncnc(Oc3ccc4[nH]c(C)cc4c3F)c2c1C,True
4120,Cc1cc2c(F)c(Oc3ncnn4cc(OCCCNS(C)(=O)=O)c(C)c34...,True
2979,COc1cc2c(Oc3ccc(N/C=C4\C(=O)NC(=O)N(c5ccc(C)cc...,True


# Hyperparameter Optimization

In [4]:
def run_svc_gridsearch(train_fps, test_fps):
    split_index = [-1] * len(train_fps) + [0] * len(test_fps)
    pds = PredefinedSplit(test_fold = split_index)

    X = train_fps + test_fps
    y = train['value'].to_list() + test['value'].to_list()

    params = {
    'C': [0.1, 0.5, 1.0, 2.0, 5.0],
    }
    svc = SVC()

    grid_search = GridSearchCV(svc, params, cv=pds, refit=False, scoring='average_precision', verbose=3)
    grid_search.fit(X, y)

    best_params = grid_search.best_params_
    print(best_params)
    svc = SVC(**best_params)
    svc.fit(train_fps, train['value'])

    test_preds = svc.predict(test_fps)
    test_metrics = get_hi_metrics(test, test_preds)
    return test_metrics


In [5]:
train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in train_mols]

test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
test_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in test_mols]

test_metrics = run_svc_gridsearch(train_morgan_fps, test_morgan_fps)
print(test_metrics)

Fitting 1 folds for each of 5 candidates, totalling 5 fits
[CV 1/1] END .............................C=0.1;, score=0.595 total time=   1.6s
[CV 1/1] END .............................C=0.5;, score=0.634 total time=   1.5s
[CV 1/1] END .............................C=1.0;, score=0.633 total time=   1.6s
[CV 1/1] END .............................C=2.0;, score=0.622 total time=   1.5s
[CV 1/1] END .............................C=5.0;, score=0.616 total time=   1.5s
{'C': 0.5}
{'roc_auc': 0.5106295280924605, 'bedroc': 0.8821163634213403, 'prc_auc': 0.5896145023811126}


# Final Evaluation

In [6]:
def fit_predict(train, test):
    train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
    train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in train_mols]

    test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
    test_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in test_mols]

    svc = SVC(
        C=0.5
    )
    svc.fit(train_morgan_fps, train['value'])

    train_result = train.copy()
    train_result['preds'] = svc.predict(train_morgan_fps)

    test_result = test.copy()
    test_result['preds'] = svc.predict(test_morgan_fps)

    return train_result, test_result


In [7]:
for i in [1, 2, 3]:
    train = pd.read_csv(f'../../../../data/hi/kdr/train_{i}.csv')
    test = pd.read_csv(f'../../../../data/hi/kdr/test_{i}.csv')

    train_preds, test_preds = fit_predict(train, test)
    train_preds.to_csv(f'../../../../predictions/hi/kdr/svc_ecfp4/train_{i}.csv')
    test_preds.to_csv(f'../../../../predictions/hi/kdr/svc_ecfp4/test_{i}.csv')
